<a href="https://colab.research.google.com/github/sarveswararao-cs/Kaggle/blob/master/aviation_fatalities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reducing Commercial Aviation Fatalities**

Loading the dataset from google drive to colab

In [1]:
!pip install h5py pyyaml
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import pandas as pd
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#train_downloaded = drive.CreateFile({'id': '1SzuFoIgicCUy-wvi3X88EnwdmehLAijd'})
#train_downloaded.GetContentFile('train.csv')
test_downloaded = drive.CreateFile({'id': '106-BUecmSzz0-PTxk-cRu_CTlkS83cLA'})
test_downloaded.GetContentFile('test.csv') 

Importing Helper libraries

In [0]:
import pandas as pd
import numpy as np

# Loading the training data from colab to Pandas DataFrame
#df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_test.shape

Viewing the top 5 rows of the training set

In [4]:
df_train.head(5)

,crew,experiment,time,seat,eeg_fp1,eeg_f7,eeg_f8,eeg_t4,eeg_t6,eeg_t5,...,eeg_c4,eeg_p4,eeg_poz,eeg_c3,eeg_cz,eeg_o2,ecg,r,gsr,event
0,1,CA,0.011719,1,-5.28545,26.775801,-9.527310,-12.793200,16.717800,33.737499,...,37.368999,17.437599,19.201900,20.5968,-3.95115,14.507600,-4520.0,817.705994,388.829987,A
1,1,CA,0.015625,1,-2.42842,28.430901,-9.323510,-3.757230,15.969300,30.443600,...,31.170799,19.399700,19.689501,21.3547,1.33212,17.750200,-4520.0,817.705994,388.829987,A
2,1,CA,0.019531,1,10.67150,30.420200,15.350700,24.724001,16.143101,32.142799,...,-12.012600,19.396299,23.171700,22.4076,1.53786,22.247000,-4520.0,817.705994,388.829987,A
3,1,CA,0.023438,1,11.45250,25.609800,2.433080,12.412500,20.533300,31.494101,...,18.574100,23.156401,22.641199,19.3367,2.54492,18.998600,-4520.0,817.705994,388.829987,A
4,1,CA,0.027344,1,7.28321,25.942600,0.113564,5.748000,19.833599,28.753599,...,6.555440,22.754700,22.670300,20.2932,1.69962,22.812799,-4520.0,817.705994,388.829987,A


Dividing the datset into data and labels

In [0]:
X = df_train.iloc[:, 0:27]
#y = df_train.iloc[:, 27]

Viewing the top 5 labels

In [0]:
#y.head(5)

Doing one hot encoding on the ["crew", 'experiment'] 

In [6]:
X_new = pd.get_dummies(X, columns = ["experiment", 'crew'])

print(X.columns)
# just clearning the X DataFrame as tensorflow session is keepon crashing 
X = None
print(X)

# converting the labels to int
y = df_train.event.astype("category").cat.codes


y.head(5)
#X_new2 = X_new.values
#labels2 = labels.values


Index(['crew', 'experiment', 'time', 'seat', 'eeg_fp1', 'eeg_f7', 'eeg_f8',
       'eeg_t4', 'eeg_t6', 'eeg_t5', 'eeg_t3', 'eeg_fp2', 'eeg_o1', 'eeg_p3',
       'eeg_pz', 'eeg_f3', 'eeg_fz', 'eeg_f4', 'eeg_c4', 'eeg_p4', 'eeg_poz',
       'eeg_c3', 'eeg_cz', 'eeg_o2', 'ecg', 'r', 'gsr'],
      dtype='object')
None


0    0
1    0
2    0
3    0
4    0
dtype: int8

In [7]:
X_new.head(5)

,time,seat,eeg_fp1,eeg_f7,eeg_f8,eeg_t4,eeg_t6,eeg_t5,eeg_t3,eeg_fp2,...,experiment_SS,crew_1,crew_2,crew_3,crew_4,crew_5,crew_6,crew_7,crew_8,crew_13
0,0.011719,1,-5.28545,26.775801,-9.527310,-12.793200,16.717800,33.737499,23.712299,-6.695870,...,0,1,0,0,0,0,0,0,0,0
1,0.015625,1,-2.42842,28.430901,-9.323510,-3.757230,15.969300,30.443600,21.010300,-6.474720,...,0,1,0,0,0,0,0,0,0,0
2,0.019531,1,10.67150,30.420200,15.350700,24.724001,16.143101,32.142799,25.431801,-0.088707,...,0,1,0,0,0,0,0,0,0,0
3,0.023438,1,11.45250,25.609800,2.433080,12.412500,20.533300,31.494101,19.142799,-0.256516,...,0,1,0,0,0,0,0,0,0,0
4,0.027344,1,7.28321,25.942600,0.113564,5.748000,19.833599,28.753599,20.572100,-1.953470,...,0,1,0,0,0,0,0,0,0,0


In [0]:
features_to_be_standardized = ['time', 'eeg_fp1', 'eeg_f7', 'eeg_f8',
       'eeg_t4', 'eeg_t6', 'eeg_t5', 'eeg_t3', 'eeg_fp2', 'eeg_o1', 'eeg_p3',
       'eeg_pz', 'eeg_f3', 'eeg_fz', 'eeg_f4', 'eeg_c4', 'eeg_p4', 'eeg_poz',
       'eeg_c3', 'eeg_cz', 'eeg_o2', 'ecg', 'r', 'gsr']

In [0]:
from sklearn.preprocessing import StandardScaler

features_data = X_new[features_to_be_standardized]
scaler = StandardScaler().fit(features_data)
standardized_features_data = scaler.transform(features_data)
X_new[features_to_be_standardized] = standardized_features_data

In [0]:
# Droping the unuseful data
features_data = None
standardized_features_data = None

In [0]:
X_new = X_new.values
y_new = y.values

Now its time to divide the training set to train and test set

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(X_new, y_new, test_size = 0.3, random_state = 1)

In [0]:
X_train.shape, type(X_train), type(y_train)

((3407194, 37), numpy.ndarray, numpy.ndarray)

In [0]:
X_test.shape. y_test.shape

In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

In [0]:
model = keras.Sequential([
    keras.layers.Dense(256, activation=tf.nn.relu),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation=tf.nn.relu),
    #keras.layers.Dropout(0.5),
    keras.layers.Dense(4, activation=tf.nn.softmax)
])

In [0]:
type(X_train)

numpy.ndarray

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Now its time to train our designed model

In [17]:
model.fit(X_train, y_train, epochs=10, batch_size = 2048, callbacks = [cp_callback])

Epoch 1/10
3395584/3407194 [============================>.] - ETA: 0s - loss: 0.2017 - acc: 0.9294
Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3407194/3407194 [==============================] - 13s 4us/step - loss: 0.2015 - acc: 0.9295
Epoch 2/10
3399680/3407194 [============================>.] - ETA: 0s - loss: 0.1288 - acc: 0.9539
Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3407194/3407194 [==============================] - 11s 3us/step - loss: 0.1288 - acc: 0.9539
Epoch 3/10
3399680/3407194 [============================>.] - ETA: 0s - loss: 0.1102 - acc: 0.9588
Epoch 00003: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3407194/3407194 [==============================] - 11s 3us/step - loss: 0.1102 - acc: 0.9588
Epoch 4/10
3397632/3407194 [============================>.] - ETA: 0s - loss: 0.0983 - acc: 0.9622
Epoch 00004: 

Evaluating the model with X_test, y_test

In [18]:
test_loss, test_acc = model.evaluate(X_test, y_test)

 

1460227/1460227 [==============================] - 78s 53us/step


In [19]:
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

Test accuracy: 0.9767864859367755
Test loss: 0.057043813959889465


## Our Saved model 

In [20]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


## Making Predictions for kaggle submission

As we have trained our model, we can start making our predictions for test data for submission to kaggle

In [0]:
predictions = model.predict()